In [1]:
from evaluation.mol_structure import mol_to_smiles, nx_to_mol
# Generate graphs for demonstration purposes
import utils.graph_generators as gen
import torch
import dgl
import networkx as nx
from rdkit.Chem import Descriptors
from evaluation.moses.metrics import get_all_metrics
from evaluation.evaluator import Evaluator
from evaluation.gin_evaluation import load_feature_extractor, MMDEvaluation
import copy
import fuckit

In [2]:
a=gen.load_zinc(n_samples=10)
grids=a
lobsters=a
#print(len(grids[0].ndata['attr']))
#print(grids[0].ndata['attr']  , grids[0].ndata['feat'])

In [4]:
lobsters_nx = [ dgl.to_networkx(g, node_attrs=['attr','feat'], edge_attrs=['attr','feat'])  for g in a]
grids_nx = [dgl.to_networkx(g, node_attrs=['attr','feat'], edge_attrs=['attr','feat']) for g in a]

#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

device=torch.device('cpu')

grids = [ dgl.from_networkx(g,node_attrs=['feat'], edge_attrs=['feat']).to(device) for g in grids_nx] # Convert graphs to DGL from NetworkX
lobsters = [ dgl.from_networkx(g,node_attrs=['feat'], edge_attrs=['feat']).to(device) for g in lobsters_nx] # Convert graphs to DGL from NetworkX

In [7]:
device=torch.device('cpu')
metrics_type=['nn', 'structural', 'molecular']
structural_statistic='nspdk'  #options ['WL' , 'nspdk', 'orbits, 
#'degree', 'clustering' ,'spectral']

reference_graphs=lobsters_nx
generated_graphs=grids_nx

discrete_node_label_name='attr' #leave it blank if this does not exist, default: 'label'
 #leave blank if this does not exist, default: 'label'
continuous_node_label_name='feat'  #leave it blank if this does not exist default: 'attr'
discrete_edge_label_name='attr'  #leave it blank if this does not exist  ,default: 'label'
continuous_edge_label_name='feat' #leave it blank if this does not exist , default: 'attr'
#dicrete labels should be set to 'label'
#continous labels should be set to 'attr'


def preprocess(nx_dataset,discrete_node_label_name, continuous_node_label_name,discrete_edge_label_name,  continuous_edge_label_name):
    processed_dataset=[]
    for G in nx_dataset:
        dict=nx.get_node_attributes(G, discrete_node_label_name)
        nx.set_node_attributes(G, dict, 'label') 
        dict=nx.get_node_attributes(G, continuous_node_label_name)
        nx.set_node_attributes(G, dict, 'attr')
        dict=nx.get_edge_attributes(G, discrete_edge_label_name)
        nx.set_node_attributes(G, dict, 'label')
        dict=nx.get_edge_attributes(G, continuous_edge_label_name)
        nx.set_node_attributes(G, dict, 'attr')
        processed_dataset.append(G)
    return processed_dataset


reference_graphs=preprocess(reference_graphs,discrete_node_label_name, continuous_node_label_name, discrete_edge_label_name,continuous_edge_label_name)
generated_graphs=preprocess(generated_graphs,discrete_node_label_name, continuous_node_label_name, discrete_edge_label_name,continuous_edge_label_name)
reference_graphs_dgl = [ dgl.from_networkx(g,node_attrs=[continuous_node_label_name], edge_attrs=[continuous_edge_label_name]).to(device) for g in reference_graphs] # Convert graphs to DGL from NetworkX
generated_graphs_dgl = [ dgl.from_networkx(g,node_attrs=[continuous_node_label_name], edge_attrs=[continuous_edge_label_name]).to(device) for g in generated_graphs] # Convert graphs to DGL from NetworkX
metrics={}
if  'nn' in metrics_type:
    assert continuous_node_label_name!='' ,'You need continuous features to be able to compure nn-based metrics'
    eval=Evaluator(feature_extractor ='gin',device='cpu')
    nn_metrics=eval.evaluate_all(generated_dataset=reference_graphs_dgl,reference_dataset=reference_graphs_dgl)
    metrics.update(nn_metrics)
        
if 'structural' in metrics_type  :
    def fun():
        eval=Evaluator(feature_extractor ='mmd-structure',device='cpu' ,statistic=structural_statistic)
        structural_metrics=eval.evaluate_all(generated_dataset=generated_graphs_dgl,reference_dataset=reference_graphs_dgl)
        metrics.update(structural_metrics)
    try:  fun()
    except: print('Cannot compute these metrics for this type of graphs')
    
if 'molecular'  in metrics_type:
    #@fuckit
    def func(reference_graphs, generated_graphs):
        reference_graphs_mol= [nx_to_mol(g,edge_label='feat', node_label='feat') for g in reference_graphs]
        generated_graphs_mol=[nx_to_mol(g,edge_label='feat', node_label='feat') for g in generated_graphs]
        reference_graphs_smiles=mol_to_smiles(reference_graphs_mol)
        generated_graphs_smiles=mol_to_smiles(generated_graphs_mol)

        mol_metrics=get_all_metrics(gen=generated_graphs_smiles,train=reference_graphs_smiles)
        metrics.update(mol_metrics)
    try: 
        func(reference_graphs,generated_graphs)
    except: 
        print('Cannot compute molecular metrics for this type of graphs. You might neeed  to manually change the definition of nx_to_mol:)\n')
        
    
print(metrics)

orthogonal
s


c:\Users\cs843\Documents\PhD\2023\February\GGM-metrics\evaluation\moses\metrics\metrics.py:216: UserWarning: Can't compute unique@1000.gen contains only 10 molecules
  warnings.warn(
c:\Users\cs843\Documents\PhD\2023\February\GGM-metrics\evaluation\moses\metrics\metrics.py:216: UserWarning: Can't compute unique@10000.gen contains only 10 molecules
  warnings.warn(


{'activations_time': 0.01700425148010254, 'fid': -4.820885820322474e-06, 'fid_time': 0.02102041244506836, 'kid': -0.5831497, 'kid_time': 0.029989957809448242, 'precision': 1.0, 'recall': 1.0, 'f1_pr': 1.00001, 'precision_time': 0.0450129508972168, 'recall_time': 0.0450129508972168, 'f1_pr_time': 0.0450129508972168, 'density': 1.2000000000000002, 'coverage': 1.0, 'f1_dc': 1.090919173552968, 'density_time': 0.034012556076049805, 'coverage_time': 0.034012556076049805, 'f1_dc_time': 0.034012556076049805, 'mmd_rbf': 0, 'mmd_rbf_time': 0.04401230812072754, 'mmd_linear': 0.0, 'mmd_linear_time': 0.01700425148010254, 'nspdk_mmd': 0.0, 'nspdk_mmd_time': 0.1530454158782959, 'valid': 1.0, 'unique@1000': 1.0, 'unique@10000': 1.0, 'FCD/Test': 46.72970354700773, 'SNN/Test': 0.49758105874061587, 'Frag/Test': 0.6078516005720516, 'Scaf/Test': 0.029266636262645807, 'FCD/TestSF': 47.06954094903616, 'SNN/TestSF': 0.4751725435256958, 'Frag/TestSF': -0.03829807449991973, 'Scaf/TestSF': 0.0, 'IntDiv': 0.77986